In [1]:
import psycopg2, psycopg2.extras, psycopg2.pool, pickle, json
from multiprocessing import Pool, Manager
from collections import OrderedDict
from shapely.geometry import mapping, shape
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
from sqlalchemy import create_engine
import pandas as pd
import geopandas as gpd
import seaborn as sns
%matplotlib inline

# Latest Tile Stats: Compare Disaster Mapping Tiles to Others

Query the `latest-tile-data-v3` database and identify tiles with specific characteristics.

In [2]:
# Connect to the database with psycopg2
d = "dbname=latest-tile-data-v3"+" user=anderstj host=127.0.0.1"
conn = psycopg2.pool.PersistentConnectionPool(1, 5, dsn=d)
cursor = conn.getconn().cursor(cursor_factory = psycopg2.extras.DictCursor)

In [3]:
#Create the engine for Pandas to query SQL with
engine = create_engine('postgresql://anderstj@127.0.0.1:5432/latest-tile-data-v3')

In [4]:
study_tiles = [
    {"name" : "Port Au Prince, Haiti", "quad" : "032211203001"},   
    {"name" : "Trisuli Bazar, Nepal",  "quad" : "123131221200"},
    {"name" : "Tacloban",              "quad" : "132312223332"},
    {"name" : "Kenema, Sierra Leone",  "quad" : "033330200220"},
    {"name" : "Monrovia, Liberia",     "quad" : "033330222101"},
    {"name" : "Kathmandu, Nepal",      "quad" : "123131221232"},
    {"name" : "Heidelberg, Germany",   "quad" : "120203320232"},
    {"name" : "London",                "quad" : "031313131103"},
    {"name" : "Berlin",                "quad" : "120210233222"},
    {"name" : "Manhattan, NY",         "quad" : "032010110132"},
    {"name" : "Denver, CO",            "quad" : "023101030121"},
]

In [9]:
# Get study_tile stats
def get_tile_df(quadkey):
    q_s = """SELECT * FROM roads, buildings, geometry WHERE 
        roads.quadkey = buildings.quadkey AND 
        geometry.quadkey = roads.quadkey AND 
        roads.quadkey = '{0}'""".format(quadkey)
    df = pd.read_sql_query(q_s,con=engine)
    
    df['named_road_ratio'] = (df.named_edited_km + df.named_new_km) / df.total_km
    df['more_building_ratio'] = (df.new_buildings_more + df.edited_buildings_more) / df.total_buildings
    
    return df

for tile in study_tiles:
    tile['characteristics'] = get_tile_df(tile['quad'])

In [13]:
def get_similar_tiles(tile, tolerance=0.01, ratio_tol=0.01):
    df = tile['characteristics']

    min_roads = (df.total_km - df.total_km*tolerance).values[0]
    max_roads = (df.total_km + df.total_km*tolerance).values[0]

    min_buildings = (df.total_buildings - df.total_buildings*tolerance).values[0]
    max_buildings = (df.total_buildings + df.total_buildings*tolerance).values[0]
    
    min_roads_ratio = (df.named_road_ratio - df.named_road_ratio*ratio_tol).values[0]
    max_roads_ratio = (df.named_road_ratio + df.named_road_ratio*ratio_tol).values[0]
    
    min_building_ratio = (df.more_building_ratio - df.more_building_ratio*ratio_tol).values[0]
    max_building_ratio = (df.more_building_ratio + df.more_building_ratio*ratio_tol).values[0]

    query_string = """SELECT * FROM roads, buildings, geometry WHERE roads.quadkey = buildings.quadkey AND geometry.quadkey = roads.quadkey AND 
    roads.total_km > {0} AND roads.total_km < {1} AND roads.quadkey != '{4}' AND 
    buildings.total_buildings > {2} AND buildings.total_buildings < {3} AND

    (roads.named_edited_km + roads.named_new_km)/roads.total_km > {5} AND 
    (roads.named_edited_km + roads.named_new_km)/roads.total_km < {6} AND

    (buildings.edited_buildings_more + buildings.new_buildings_more)::float / buildings.total_buildings::float > {7} AND
    (buildings.edited_buildings_more + buildings.new_buildings_more)::float / buildings.total_buildings::float < {8}

    """.format(min_roads, max_roads, min_buildings, max_buildings, df.quadkey.values[0][0], min_roads_ratio, max_roads_ratio, min_building_ratio, max_building_ratio)
#     print(query_string)
    return pd.read_sql_query(query_string,con=engine)

x = get_similar_tiles(study_tiles[2], tolerance = 0.25, ratio_tol=0.25)
print(len(x))
x.head()

1


,quadkey,named_edited_km,named_new_km,total_edited_km,total_km,total_new_km,unnamed_edited_km,unnamed_new_km,quadkey,total_buildings,total_new_buildings,total_edited_buildings,new_buildings_more,new_buildings_yes,edited_buildings_more,edited_buildings_yes,quadkey,coordinates,type
0,132303033313,99.9365,3.75274,218.701,319.301,100.6,118.764,96.8474,132303033313,27667,3366,24301,1170,2196,16127,8174,132303033313,"[[[120.849609375, 14.093957177836227], [120.84...",Polygon


In [22]:
for tile in study_tiles:
    print(tile['name'])
    sim_tiles = get_similar_tiles(tile, tolerance=0.35, ratio_tol=0.25)
    print(len(sim_tiles))
    tile['similar_tiles'] = sim_tiles

Port Au Prince, Haiti
17
Trisuli Bazar, Nepal
4
Tacloban
2
Kenema, Sierra Leone
1
Monrovia, Liberia
0
Kathmandu, Nepal
1
Heidelberg, Germany
42
London
15
Berlin
14
Manhattan, NY
7
Denver, CO
1


In [315]:
res_coll = mapping(convert_to_gpd(z).geometry)
for res in res_coll['features']:
    res['geometry']['type'] = 'MultiPoint'
    res['geometry']['coordinates'] = res['geometry']['coordinates'][0]
    del res['bbox']
    del res['id']

print(json.dumps(res_coll))

{"features": [{"properties": {}, "type": "Feature", "geometry": {"type": "MultiPoint", "coordinates": [[3.076171875, 50.625073063414355], [3.076171875, 50.68079714532165], [3.1640625, 50.68079714532165], [3.1640625, 50.625073063414355], [3.076171875, 50.625073063414355]]}}, {"properties": {}, "type": "Feature", "geometry": {"type": "MultiPoint", "coordinates": [[2.4609375, 48.748945343432936], [2.4609375, 48.80686346108518], [2.548828125, 48.80686346108518], [2.548828125, 48.748945343432936], [2.4609375, 48.748945343432936]]}}, {"properties": {}, "type": "Feature", "geometry": {"type": "MultiPoint", "coordinates": [[2.28515625, 48.748945343432936], [2.28515625, 48.80686346108518], [2.373046875, 48.80686346108518], [2.373046875, 48.748945343432936], [2.28515625, 48.748945343432936]]}}, {"properties": {}, "type": "Feature", "geometry": {"type": "MultiPoint", "coordinates": [[2.197265625, 48.980216985374994], [2.197265625, 49.03786794532643], [2.28515625, 49.03786794532643], [2.28515625, 

In [222]:
def convert_to_gpd(df):
    df['geometry'] = df.coordinates.apply(lambda coords: shape({"type":"Polygon", "coordinates":json.loads(coords)}))
    return gpd.GeoDataFrame(df)
y = convert_to_gpd(x)
y

,quadkey,named_edited_km,named_new_km,total_edited_km,total_km,total_new_km,unnamed_edited_km,unnamed_new_km,quadkey,total_buildings,...,total_edited_buildings,new_buildings_more,new_buildings_yes,edited_buildings_more,edited_buildings_yes,quadkey,coordinates,type,geometry,geojson
0,031333110213,187.39100,44.13020,220.648,281.186,60.5379,33.2572,16.4077,031333110213,27542,...,23838,2,3702,20,23818,031333110213,"[[[-1.142578125, 44.71551373202134], [-1.14257...",Polygon,"POLYGON ((-1.142578125 44.71551373202134, -1.1...","{'type': 'Polygon', 'coordinates': [[[-1.14257..."
1,310013121223,3.66180,0.00000,140.529,240.788,100.2590,136.8670,100.2590,310013121223,28685,...,25,0,28660,0,25,310013121223,"[[[110.478515625, -7.710991655433223], [110.47...",Polygon,"POLYGON ((110.478515625 -7.710991655433223, 11...","{'type': 'Polygon', 'coordinates': [[[110.4785..."
2,300132102231,8.39388,3.58473,185.854,259.588,73.7340,177.4600,70.1493,300132102231,27350,...,951,1397,25002,140,811,300132102231,"[[[36.826171875, -17.895114303749146], [36.826...",Polygon,"POLYGON ((36.826171875 -17.89511430374915, 36....","{'type': 'Polygon', 'coordinates': [[[36.82617..."
3,030220003002,91.99230,125.46400,114.282,278.608,164.3270,22.2893,38.8626,030220003002,29294,...,103,1,29190,3,100,030220003002,"[[[-89.296875, 48.341646172374595], [-89.29687...",Polygon,"POLYGON ((-89.296875 48.3416461723746, -89.296...","{'type': 'Polygon', 'coordinates': [[[-89.2968..."
4,210312313010,121.34300,16.96900,191.144,278.217,87.0723,69.8017,70.1033,210312313010,27976,...,7,1005,26964,4,3,210312313010,"[[[-51.15234375, -30.221101852485987], [-51.15...",Polygon,"POLYGON ((-51.15234375 -30.22110185248599, -51...","{'type': 'Polygon', 'coordinates': [[[-51.1523..."


In [221]:
with open('tmp.geojson','w') as oFile:
    oFile.write(y.to_json())

In [92]:
roads = 0.5
num_buildings = 0
buildings = 0.5

#roads.quadkey, buildings.total_buildings, ((roads.named_edited_km + roads.named_new_km)/roads.total_km), (buildings.edited_buildings_more + buildings.new_buildings_more)::float / buildings.total_buildings::float

QUERY_STRING = """SELECT count(*) FROM roads, buildings, geometry WHERE roads.quadkey = buildings.quadkey AND geometry.quadkey = roads.quadkey AND 
    roads.total_km > 0 AND 
        (roads.named_edited_km + roads.named_new_km)/roads.total_km > {0} AND 
    buildings.total_buildings > {1} AND 
        (buildings.edited_buildings_more + buildings.new_buildings_more)::float / buildings.total_buildings::float > {2}
LIMIT 10;""".format(roads, num_buildings, buildings)
print(QUERY_STRING)

SELECT count(*) FROM roads, buildings, geometry WHERE roads.quadkey = buildings.quadkey AND geometry.quadkey = roads.quadkey AND 
    roads.total_km > 0 AND 
        (roads.named_edited_km + roads.named_new_km)/roads.total_km > 0.5 AND 
    buildings.total_buildings > 0 AND 
        (buildings.edited_buildings_more + buildings.new_buildings_more)::float / buildings.total_buildings::float > 0.5
LIMIT 10;


In [93]:
df = pd.read_sql_query(QUERY_STRING,con=engine)

In [94]:
df.head()

,count
0,13177


In [11]:
cursor.execute("SELECT * from roads, buildings, amenities WHERE\
               roads.total_km > 0 AND (roads.named_edited_km + roads.named_new_km)/roads.total_km > %s \
                LIMIT 10;",(roads, ))